In [2]:
import sqlalchemy
import pandas as pd
from wb_api_wraper import *

In [4]:
engine = sqlalchemy.create_engine('mssql+pyodbc://wbmserccar101/POVANA14')

## Lists all tables 

In [7]:
#gtap maping file


In [5]:
conso_gtap=pd.read_sql_query("SELECT * FROM gtap9..basedata where header in ('vdpm','vipm')", engine) #value of prdivate conso imported Domestic (Market) prices 
#value in million usd mer 2011 (gtap 9)

DBAPIError: (Error) ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied. (17) (SQLDriverConnect)') None None

In [9]:
conso_gtap.head()

,header,element1,element2,element3,element4,value
0,VDPM,atp,alb,None,None,48.559628
1,VDPM,atp,are,None,None,1493.457000
2,VDPM,atp,arg,None,None,1414.448500
3,VDPM,atp,arm,None,None,12.545170
4,VDPM,atp,aus,None,None,4395.786100


In [10]:
dico = {'element1':'cpc2','element2':'gtap_region','value':'conso_usd','header':'source'}
list(dico.values())
# indx = ["gtap_region","cpc2",'source']
indx = ["gtap_region","cpc2"]

In [11]:
df=pd.DataFrame()
#here i am just summing international and  domestic consumption...
a=conso_gtap.rename(columns=dico)[list(dico.values())].groupby(["gtap_region","cpc2"]).sum() 
df=pd.concat([df,a])

gtap_emi_intensity = pd.read_sql_query("SELECT * FROM emissions..hhemissions", engine) #emissions directly from GTAP CO@ emis from FF con
b=gtap_emi_intensity.rename(columns={'commodity':'cpc2','country':'gtap_region'}).set_index(indx).sortlevel()
df = pd.concat([df,b],axis=1)

In [12]:
gtap_to_wb = pd.read_csv("gtap_to_wb_name.csv",index_col='gtap_region', squeeze=True)
iso_to_wb=pd.read_csv("iso3_to_wb_name.csv",index_col="iso3",squeeze=True)

In [13]:
df["emissions_g"] = df["conso_usd"]*df["emissions_g_usd"]
tot_emis_gtap =df.emissions_g.sum(level="gtap_region").reset_index()
tot_emis_gtap["country"]= (tot_emis_gtap.gtap_region.str.strip().str.upper()).replace(gtap_to_wb).replace(iso_to_wb)
tot_emis_gtap = tot_emis_gtap.set_index("country")["emissions_g"]
tot_emis_gtap.to_csv("tot_emis_gtap.csv")

In [1]:
df

NameError: name 'df' is not defined

In [14]:
wb_emis = get_wb_mrv("EN.ATM.CO2E.FF.KT","wb_emis") #CO2 emissions from fossil fuels, total;


In [15]:
e=pd.DataFrame()


In [6]:
(wb_emis/tot_emis_gtap).dropna()

NameError: name 'wb_emis' is not defined

In [17]:
gtap_to_cpc=pd.read_sql_query("SELECT * FROM cpc..GTAP_CPC2", engine) #gtap reports in ISIC and CPC1 but Maros and co tranlstated everything to CPC2
gtap_to_cpc

,GTAP,CPC
0,ATP,64134
1,ATP,64241
2,ATP,64242
3,ATP,64243
4,ATP,64244
5,ATP,64250
6,ATP,65311
7,ATP,65319
8,ATP,65320
9,ATP,66031


In [18]:
a=pd.read_sql_query("SELECT * FROM information_schema.tables where table_schema='combined'", engine)
a

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,POVANA14,combined,tradedatagtp,BASE TABLE
1,POVANA14,combined,strata,BASE TABLE
2,POVANA14,combined,emissionsbyhousehold,BASE TABLE
3,POVANA14,combined,PovertyLines,BASE TABLE
4,POVANA14,combined,tradedatafood,BASE TABLE
5,POVANA14,combined,PerCapitaExpenditures,BASE TABLE
6,POVANA14,combined,HouseholdsRanked,BASE TABLE
7,POVANA14,combined,PERSDATA,BASE TABLE
8,POVANA14,combined,HHLDDATA,BASE TABLE
9,POVANA14,combined,cpe,BASE TABLE


###Downloads those I need

In [19]:
import os
os.makedirs("csvdata",exist_ok=True)

In [20]:
#pd.read_sql_table('data', engine)
pd.read_sql_table("emissionsbyhousehold",engine,schema="combined").to_csv("csvdata/emissionsbyhousehold.csv")
pd.read_sql_table("PerCapitaExpenditures",engine,schema="combined").to_csv("csvdata/PerCapitaExpenditures.csv")
pd.read_sql_table("HHLDDATA",engine,schema="combined").to_csv("csvdata/HHLDDATA.csv"             )
pd.read_sql_table("tradedatagtp",engine,schema="combined").to_csv("csvdata/tradedatagtp.csv"         )
pd.read_sql_table("PERSDATA",engine,schema="combined").to_csv("csvdata/PERSDATA.csv"             )
pd.read_sql_table("DecileComposite",engine,schema="combined").to_csv("csvdata/DecileComposite.csv"      )